MNIST 는 0 ~ 9 까지 10 종류의 손글씨 숫자로 이뤄진 데이터셋입니다. 이미지 분류의 대표적인 toy data 로 이용되고 있습니다. Fashion-MNIST 는 10 종류의 의류관련 이미지들을 모아둔 데이터셋입니다. 이미지는 28 x 28 크기의 흑백이미지로, 손글씨 숫자데이터 MNIST 와 같은 형식입니다. 이 데이터는 아래의 github repository 를 복사하여 이용할 수 있습니다.

https://github.com/zalandoresearch/fashion-mnist

Repository 내의 `data/fashion/` 폴더에는 압축된 형식으로 총 7만장의 10종류의 이미지가 포함되어 있으며, `utils/` 폴더에는 이 데이터를 로딩하는 유틸함수가 포함되어 있습니다. 그런데 이 폴더의 함수를 import 하기 위해서는 다운로드 받은 fashion-MNIST 의 위치를 등록해야 합니다.

In [1]:
%matplotlib inline

import sys
mnist_utils_path = '../../../../fashion-mnist/utils/'
mnist_data_path  = '../../../../fashion-mnist/data/fashion/'

`sys.path` 에는 `import` 할 수 있는 패키지들의 위치가 저장되어 있습니다. 설치되지 않은 (다운로드만 받은) 패키지를 이용하기 위해서는 `sys.path` 에 해당 패키지의 위치를 등록해야 합니다.

In [2]:
# sys.path

`sys.path` 는 Python 의 `list of str` 형식입니다. 여기에 위의 `str` 값을 추가해 봅니다.

In [3]:
sys.path.append(mnist_utils_path)

In [4]:
# sys.path

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='poster')

학습용 데이터와 테스트용 데이터를 모두 불러옵니다.

In [6]:
from mnist_reader import load_mnist

train, train_labels = load_mnist(mnist_data_path, kind='train')
test, test_labels   = load_mnist(mnist_data_path, kind='t10k')

print(train.shape)
print(train_labels.shape)
print(test.shape)
print(test_labels.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


데이터는 numpy.ndarray 로 정제되어 있습니다. 이는 28 x 28 의 이미지가 flatten 된 형식입니다.

In [7]:
print(type(train[0]))
print(train[0].shape)

<class 'numpy.ndarray'>
(784,)


PIL 은 Python Image Library 의 약자로, 이미지 처리를 도와주는 패키지입니다. `fromarray()` 함수를 이용하면 numpy.ndarray 를 이미지로 변환해줍니다. 첫번째 이미지는 신발입니다.

In [8]:
from PIL import Image

image = Image.fromarray(train[0].reshape(28,28))
image

두번째 이미지는 티셔츠입니다.

In [9]:
image = Image.fromarray(train[1].reshape(28,28))
image

반대로 numpy.narray 안에 image 를 입력하면 numpy.ndarray 로 변환됩니다.

In [10]:
np.array(image).shape

(28, 28)

`train` 데이터의 type 이 `uint8` 입니다. 이는 unsigned 8-bit integer 라는 의미로, 0 부터 2^8 -1, 즉 0 ~ 255 까지의 integer 값을 가집니다. 참고로 8-bit integer 는 -2^7 부터 2^7 - 1 까지, 즉 -128 ~ 127 까지 표현가능 합니다. RBG color 는 음수로 표현할 필요가 없기 때문에 `uint` 를 이용하여 주로 표현합니다.

In [11]:
train.dtype

dtype('uint8')

데이터는 [0, 255] 의 값으로 표현된 흑백이미지 입니다. 이를 [0, 1] 로 scaling 합니다. 또한 float-64bit 로 타입도 변환 (casting) 합니다. 머신러닝 알고리즘들은 주로 float 에서 작동하도록 설계되어 있으며 많은 알고리즘들이 `unit` 를 지원하지 않습니다.

In [12]:
data = np.array(np.vstack([train, test]), dtype=np.float64) / 255.0
target = np.hstack([train_labels, test_labels])

Scikit-learn 의 multi-layer perceptron classifier 를 이용하여 이를 분류하는 모델을 학습해 봅니다. 학습 및 성능 평가를 반복할 것이기 때문에 이 과정 역시 함수로 만듭니다. Python 의 time package 를 이용하면 현재 시각을 UNIX time (1970.01.01 00:00:00 으로부터의 seconds) 으로 표현할 수 있습니다. 이 시간의 차이를 계산하여 학습 및 테스트 시간을 측정합니다. 그리고 classification_report 함수를 이용하여 클래스 별 성능을 확인합니다.

In [13]:
from time import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn import metrics


def train_and_evaluate(model, scaler, X_train, y_train, X_test, y_test):
    # training
    t = time()
    X_train_scaled = scaler.fit_transform(X_train)
    model.fit(X_train_scaled, y_train)
    t = time() - t
    print(f'training time = {t:.3} secs. with n={X_train.shape[0]}')

    # test
    t = time()
    X_test_scaled = scaler.transform(X_test)
    y_test_pred = model.predict(X_test_scaled)
    t = time() - t
    print(f'test     time = {t:.3} secs. with n={X_test.shape[0]}\n')
    print(metrics.classification_report(y_test, y_test_pred))

    return model

학습데이터는 클래스의 순서가 뒤섞여 있는 데이터입니다.

In [14]:
train_labels[:10]

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5], dtype=uint8)

우선 3000 개의 데이터만 이용하여 모델을 학습합니다. hidden layer size, activation function, optimizer 등을 설정합니다. `warm_start = True` 로 설정하면 모델이 충분히 학습되지 않았을 때 현재 weights 를 초기값으로 추가학습을 할 수 있습니다.

In [15]:
model = MLPClassifier(
    hidden_layer_sizes = (200, 50),
    activation = 'relu',
#     solver = 'sgd',
    solver = 'adam',
    warm_start = True,
    max_iter = 1000,
    epsilon = 0.00001
)
scaler = StandardScaler()

n_samples = 3000
X = train[:n_samples]
y = train_labels[:n_samples]

model = train_and_evaluate(model, scaler, X, y, test, test_labels)

training time = 8.38 secs. with n=3000
test     time = 0.253 secs. with n=10000

              precision    recall  f1-score   support

           0       0.80      0.78      0.79      1000
           1       0.97      0.95      0.96      1000
           2       0.71      0.75      0.73      1000
           3       0.83      0.85      0.84      1000
           4       0.73      0.75      0.74      1000
           5       0.93      0.89      0.91      1000
           6       0.60      0.57      0.58      1000
           7       0.89      0.92      0.90      1000
           8       0.95      0.92      0.93      1000
           9       0.91      0.93      0.92      1000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



성능이 아쉽습니다. 이번에는 학습데이터를 20000 개로 늘려서 추가학습을 합니다. 그런데 데이터의 개수가 증가하였음에도 불구하고 학습 시간이 매우 짧습니다. 이는 모델이 이미 weights 가 수렴했다고 판단했기 때문입니다.

In [16]:
n_samples = 20000
X = train[:n_samples]
y = train_labels[:n_samples]

model = train_and_evaluate(model, scaler, X, y, test, test_labels)

training time = 2.67 secs. with n=20000
test     time = 0.227 secs. with n=10000

              precision    recall  f1-score   support

           0       0.81      0.78      0.79      1000
           1       0.96      0.96      0.96      1000
           2       0.74      0.78      0.76      1000
           3       0.81      0.89      0.85      1000
           4       0.75      0.78      0.76      1000
           5       0.94      0.91      0.93      1000
           6       0.66      0.56      0.60      1000
           7       0.92      0.89      0.91      1000
           8       0.95      0.95      0.95      1000
           9       0.89      0.95      0.92      1000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



만약 `warm_start=False` 로 설정하면 re-initialize 되어 모델이 다시 학습됩니다. 성능이 조금이지만 개선되었습니다. 아마도 처음 3000 개의 데이터로 학습한 모델은 성능이 좋지 않은 local optima 에 갇혀있는 것이라 예상됩니다. 하지만 여전히 몇몇 클래스에 대해서는 성능이 아쉽습니다. 모델의 구조 (hidden layer size) 를 변경하던지 혹은 image classification 에 좋다고 알려진 convolutional neural network 를 이용해 보아야 할 것 같습니다.

In [17]:
n_samples = 20000
X = train[:n_samples]
y = train_labels[:n_samples]

model.warm_start = False
model = train_and_evaluate(model, scaler, X, y, test, test_labels)

training time = 46.5 secs. with n=20000
test     time = 0.271 secs. with n=10000

              precision    recall  f1-score   support

           0       0.79      0.83      0.81      1000
           1       0.99      0.96      0.97      1000
           2       0.78      0.79      0.78      1000
           3       0.86      0.86      0.86      1000
           4       0.77      0.79      0.78      1000
           5       0.95      0.94      0.94      1000
           6       0.69      0.65      0.67      1000
           7       0.94      0.93      0.93      1000
           8       0.96      0.96      0.96      1000
           9       0.93      0.96      0.94      1000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



Feed forward neural network 의 패러매터들을 살펴봅니다. 이를 위하여 여러 개의 iterable 한 객체 내 데이터를 살펴보는 zip 함수를 이용합니다. zip 함수에 두 개 이상의 list, set 등을 입력하면 길이가 짧은 객체 탐색이 끝날 때까지 각 객체 내 데이터를 함께 yield 합니다.

In [18]:
# Python zip
l1 = list('abcde')
l2 = list(range(3,6))

print(f'l1 = {l1}')
print(f'l2 = {l2}\n')

for item1, item2 in zip(l1, l2):
    print(f'item1 = {item1}, item2 = {item2}')

l1 = ['a', 'b', 'c', 'd', 'e']
l2 = [3, 4, 5]

item1 = a, item2 = 3
item1 = b, item2 = 4
item1 = c, item2 = 5


enumerate 함수는 데이터와 해당 데이터의 index 를 함께 출력합니다.

In [19]:
# Python enumerate
for i, l1 in enumerate(l1):
    print(f'#{i} = {l1}')

#0 = a
#1 = b
#2 = c
#3 = d
#4 = e


이를 이용하여 모델의 패러매터들의 크기를 살펴봅니다. 이 모델은 784 -> 200 -> 50 으로 hidden representation 을 변경한 뒤, 10 개의 클래스로 구분하는 softmax regression 을 수행합니다. 이에 해당하는 weights 와 intercept 가 다음의 크기로 학습되었습니다.

In [20]:
print(f'num layers = {len(model.coefs_)}')

coefs = model.coefs_
intercepts = model.intercepts_

for l, (coef, intercept) in enumerate(zip(coefs, intercepts)):
    print(f'layer#{l+1}: coef={coef.shape}, intercept={intercept.shape}')

num layers = 3
layer#1: coef=(784, 200), intercept=(200,)
layer#2: coef=(200, 50), intercept=(50,)
layer#3: coef=(50, 10), intercept=(10,)


In [21]:
coefs[0][:3,:3]

array([[ 0.03990664, -0.00558187,  0.04853701],
       [ 0.08314563,  0.11646285, -0.14838729],
       [-0.04296912,  0.03655537,  0.0515885 ]])

학습한 모델을 저장하거나 불러와 사용할 수 있습니다. 우선 이런 궁금증이 생긴다면 다음처럼 웹검색을 해보십시요.

```
scikit learn save model
```

검색 결과 https://scikit-learn.org/stable/modules/model_persistence.html 같은 문서를 찾았습니다. `wb` 는 write as binary 의 약어입니다. 텍스트처럼 읽히는 정보가 아니라 0/1 로 기록한다는 의미입니다.

In [22]:
import pickle

with open('./fashion_mnist_feed_forward_sklearn.pkl', 'wb') as f:
    pickle.dump(model, f)

반대로 `rb` 는 read binary file 의 약어입니다. 위와 동일한 구조와 패러매터를 가진 모델이 로딩되었습니다.

In [24]:
with open('./fashion_mnist_feed_forward_sklearn.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

coefs = loaded_model.coefs_
intercepts = loaded_model.intercepts_

for l, (coef, intercept) in enumerate(zip(coefs, intercepts)):
    print(f'layer#{l+1}: coef={coef.shape}, intercept={intercept.shape}')

print(coefs[0][:3,:3])

layer#1: coef=(784, 200), intercept=(200,)
layer#2: coef=(200, 50), intercept=(50,)
layer#3: coef=(50, 10), intercept=(10,)
[[ 0.03990664 -0.00558187  0.04853701]
 [ 0.08314563  0.11646285 -0.14838729]
 [-0.04296912  0.03655537  0.0515885 ]]


위 문서에서는 `pickle` 대신 `joblib` 을 이용하라 권합니다. 사용법은 `pickle` 과 같습니다.

In [27]:
from joblib import dump, load

dump(model, './fashion_mnist_feed_forward_sklearn.joblib')
loaded_model2 = load('./fashion_mnist_feed_forward_sklearn.joblib')
print(loaded_model2.coefs_[0][:3,:3])

[[ 0.03990664 -0.00558187  0.04853701]
 [ 0.08314563  0.11646285 -0.14838729]
 [-0.04296912  0.03655537  0.0515885 ]]
